In [20]:
import torch
import cv2
import numpy as np
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel, AutoencoderKL
from PIL import Image
import os
torch.cuda.empty_cache()

In [14]:
# Load SDXL ControlNet models (Depth & Normal)
controlnet_depth = ControlNetModel.from_pretrained(
    "diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16
)
controlnet_normal = ControlNetModel.from_pretrained(
    "xinsir/controlnet-union-sdxl-1.0", torch_dtype=torch.float16
)

controlnet_edge = ControlNetModel.from_pretrained(
    "xinsir/controlnet-union-sdxl-1.0", torch_dtype=torch.float16  # Edge detection (Canny)
)

# Load SDXL pipeline with ControlNet
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=[controlnet_depth, controlnet_normal,controlnet_edge],  # Using multiple ControlNets
    torch_dtype=torch.float16
).to("cuda")

pipe.vae = AutoencoderKL.from_pretrained(
    "stabilityai/sdxl-vae", torch_dtype=torch.float16
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

Some weights of the model checkpoint were not used when initializing ControlNetModel: 
 ['control_add_embedding.linear_1.bias, control_add_embedding.linear_1.weight, control_add_embedding.linear_2.bias, control_add_embedding.linear_2.weight, spatial_ch_projs.bias, spatial_ch_projs.weight, task_embedding, transformer_layes.0.attn.in_proj_bias, transformer_layes.0.attn.in_proj_weight, transformer_layes.0.attn.out_proj.bias, transformer_layes.0.attn.out_proj.weight, transformer_layes.0.ln_1.bias, transformer_layes.0.ln_1.weight, transformer_layes.0.ln_2.bias, transformer_layes.0.ln_2.weight, transformer_layes.0.mlp.c_fc.bias, transformer_layes.0.mlp.c_fc.weight, transformer_layes.0.mlp.c_proj.bias, transformer_layes.0.mlp.c_proj.weight']
Some weights of the model checkpoint were not used when initializing ControlNetModel: 
 ['control_add_embedding.linear_1.bias, control_add_embedding.linear_1.weight, control_add_embedding.linear_2.bias, control_add_embedding.linear_2.weight, spatial_ch_pr

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
# Function to load and preprocess an RGB normal map
def load(image_path):
    normal = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Load as RGB
    normal = cv2.cvtColor(normal, cv2.COLOR_BGR2RGB)  # Convert to RGB (OpenCV loads as BGR)
    return Image.fromarray(normal)

def apply_canny_edge_detection(path, low_threshold=100, high_threshold=200):
    """Convert an image to a Canny edge map."""
    image = Image.open(path)
    image = np.array(image.convert("L"))  # Convert to grayscale
    edges = cv2.Canny(image, low_threshold, high_threshold)
    return Image.fromarray(edges)


for i in os.listdir("bear/o"):
    depth_map_path = "bear/s/" + i  # Replace with your depth map
    normal_map_path = "bear/n/" + i  # Replace with your normal map
    edge_map_path = "bear/o/" + i
    
    depth_map = load(depth_map_path)
    normal_map = load(normal_map_path)
    edge_map = apply_canny_edge_detection(edge_map_path)
    
    # Define relighting prompt
    prompt = "A hyper-realistic polar bear standing in a tundra forest with cloudy sky and snow on the ground."
    # Generate the relit scene
    output = pipe(
        prompt=prompt,
        image=[depth_map, normal_map,edge_map],  # Control images
        num_inference_steps=70,
        controlnet_conditioning_scale=[0.9, 0.7, 0.7], # Depth 
        guidance_scale=8
    )
    
    # Save and show result
    output_image = output.images[0]
    output_image.save("bear_n/"+i+".jpg")
    #output_image.show()

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
